<a href="https://colab.research.google.com/github/Pavun-KumarCH/CNN-Classifier-End-to-end/blob/main/Medical_Inventory_Management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<h1><center><strong><font size="6">Medical Inventory Management Project<font><strong></center></h1>

## **CRISP - ML(Q)**

**CRISP-ML(Q) process model describes six phases:**

- Business and Data Understanding
- Data Preparation (Data Engineering)
- Model Building (Machine Learning)
- Model Evaluation and Tunning
- Deployment
- Monitoring and Maintenance

---

## **Problem Statements**:
Bounce rate is increasing significantly leading to patient dissatisfaction.

---

### **Business Objective** :
Minimize Bounce Rate.

### **Business Constraints** :
Minimize Inventory Cost.

---
          
## **Success Criteria** : -

**Business Success Criteria** : Reduce bounce rate by at least 30%

**Machine Learning Success Criteria** : Achieve an Accuracy of at least 90%

**Economic Success Criteria** : Increase revenue by at least 20 lacs INR by reducing bounce rate.

---

### **Data Collection** :
                  
Data Was Provided client which One of the Leading Pharma Company in india.


## **Data Description** :
The dataset consists of 14218 entries with the following columns:

**VARIABLE NAME - DESCRIPTION**

---
1. **Typeofsales** :	*Type of sale of the drug. Either the drug is sold or returned.*

2. **Patient_ID** : 	*ID of a patient*

3. **Specialisation** :	*Name of Specialisation (eg. Cardiology)*

4. **Dept** :	        *Pharmacy, the formulation is related with.*

5. **Dateofbill** :  	*Date of purchase of medicine*

6. **Quantity** :	    *Quantity of the drug*

7. **ReturnQuantity** :	*Quantity of drug returned by patient to the pharmacy*

8. **Final_Cost** :	    *Final Cost of the drug (Quantity included)*

9. **Final_Sales** :	*Final sales of drug*

10. **RtnMRP** :	       * MRP of returned drug (Quantity included)*

11. **Formulation** :	*Type of formulation*

12. **DrugName** :	    *Generic name of the drug*

13. **SubCat** :	        *Subcategory (Type) to the category of drugs*

14. **SubCat1** :     	*Subcategory (condition) to the category of drugs*


In [ ]:
# Import required libraries
import io
import pylab
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats


In [ ]:
# Suppress Warning
import warnings
warnings.filterwarnings('ignore')

### Load The Data

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
filename = next(iter(uploaded))
filename

In [ ]:
data = pd.read_excel(io.BytesIO(uploaded[filename]))


In [ ]:
data

# Financial Metric's analysis

Based on the columns 'Quantity', 'Final_Cost', 'Final_Sales', and 'RtnMRP', we can calculate various metrics that provide insights into sales data. Here are some additional calculations we perform:


---


##Total Revenue:
    This is the same as Final_Sales.


---


##Total Cost:
    This is the same as Final_Cost.


---


##Total Profit:
    Calculated as Final_Sales - Final_Cost.


---


##Profit Margin:

    Calculated as (Final_Sales - Final_Cost) / Final_Sales. This represents the percentage of revenue that turns into profit.


---


##Return on Investment (ROI):

    Calculated as (Final_Sales - Final_Cost) / Final_Cost. This measures the profitability of the investment.


---


##Average Selling Price (ASP):

    Calculated as Final_Sales / Quantity. This represents the average price at which each unit is sold.


---


##Average Cost Price (ACP):

    Calculated as Final_Cost / Quantity. This represents the average cost of each unit.


---


##Markup:

    Calculated as (Final_Sales - Final_Cost) / Final_Cost. This represents the percentage increase over the cost price.


---


##Return Margin:

    Calculated as (RtnMRP - Final_Cost) / RtnMRP. This indicates the margin when considering the maximum retail price (MRP) for returns.


---


##Revenue per Quantity:

    Calculated as Final_Sales / Quantity. This is similar to ASP but ensures clarity.


In [ ]:
# Create a DataFrame to store all related Sales Metrics
sales_df = pd.DataFrame(columns = ['Profit'])

# Calculate Total Profit
sales_df['Profit'] = pd.DataFrame(data['Final_Sales'] - data['Final_Cost'])

# Calculate Profit Margin
sales_df['Profit Margin'] = sales_df['Profit'] / data['Final_Sales']

# Calculate Return on Investment(ROI)
sales_df['ROI'] = (data['Final_Sales'] - data['Final_Cost'])/ data['Final_Cost']

# Calculate Average Selling Price(ASP)
sales_df['ASP'] = data['Final_Sales']/ data['Quantity']

# Calculate Avreage Cost Price(ACP)
sales_df['ACP'] = data['Final_Cost']/ data['Quantity']

# Calculate Profit per Unit
sales_df['Profit per Unit'] = sales_df['Profit'] / data['Quantity']

# Calculate Markup
sales_df['Mark up'] = sales_df['Profit'] / data['Final_Cost']

# Calculate Return Mark up
sales_df['Return Margin'] = (data['RtnMRP'] - data['Final_Cost'])/  data['RtnMRP']


print(f"The Total Profit sales: {sales_df['Profit'].sum()}\n")

sales_df.describe()

In [ ]:
sales_df.head()

### Retrive the Data Info

In [ ]:
data.info()

In [ ]:
data.head()

## **Data Preparation**

### Duplicates Handling


In [ ]:
duplicates = data.duplicated()
sum(duplicates)

In [ ]:
data.drop_duplicates(inplace = True)
duplicates = data.duplicated()
sum(duplicates)

### Handling Missing Values

In [ ]:
data.isna().sum()


#### Imputation

In [ ]:
# Reset the index before applying group-wise mode
data.reset_index(drop = True, inplace = True)

In [ ]:
# Impute missing values in Formulation column based on the mode of the group
group_cols = ['Typeofsales','Specialisation','Dept']

for col in ['Formulation', 'DrugName', 'SubCat', 'SubCat1']:
    data[col] = data.groupby(group_cols)[col].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x)

data.isna().sum()

In [ ]:
# We still have few missing values
data.dropna(inplace = True)
data.reset_index(drop = True, inplace = True)
data.isna().sum()

### Data Manupulation

In [ ]:
data['Dateofbill'] = pd.to_datetime(data['Dateofbill'])

# Sort the datadet based on date column in ascending order
data = data.sort_values(by = 'Dateofbill', ascending = True)

In [ ]:
# Converting date format to month
data['Months'] = data['Dateofbill'].dt.strftime("%b")
data['Week'] = data['Dateofbill'].dt.isocalendar().week
data.head(10)

In [ ]:
plt.plot(data['Week'])

In [ ]:
# Speifying columns Final cost and final sale  to round
data['Final_Cost'] = data['Final_Cost'].map(lambda x : round(x))
data['Final_Sales'] = data['Final_Sales'].map(lambda x : round(x))

### Feature Selection

In [ ]:
# droping Irrelavent columns
data.drop(['Patient_ID','ReturnQuantity'], axis = True, inplace = True)

In [ ]:
data.head(10)

### Descriptive Analytics

In [ ]:
data.describe()

## **Segregate Numeric and Non numeric columns**


In [ ]:
numeric_features = data.select_dtypes(exclude = ['object','datetime64']).columns
numeric_features

In [ ]:
categorical_features = data.select_dtypes(include = ['object']).columns
categorical_features

## First Moment Decision - Measure Of Central Tendency

We see that "MULTIPLE ELECTROLYTES 500ML IVF"	 is the top performing drug

In [ ]:
# Mean
data[numeric_features].mean()

In [ ]:
# Median
data[numeric_features].median()

In [ ]:
# Mode
data.mode()

## Second Moment Bussiness Decision - Measure of Dispersion

In [ ]:
# Variance
data[numeric_features].var()

In [ ]:
# Standard Deviation
data[numeric_features].std()

## Third Moment Business Decision - Skewness

In [ ]:
# Skewness
data[numeric_features].skew()

## Forth Moment Business Decision - Kurtosis

In [ ]:
# Kurtosis
data[numeric_features].kurt()

## Exploratory Data Analysis

In [ ]:
data.Quantity.max()

In [ ]:
data['Quantity'].plot(kind = 'hist', color = 'orange', bins = 20, alpha = 1)
plt.xlim(0, 160)

In [ ]:
data.Final_Cost.max()

In [ ]:
data['Final_Cost'].plot(kind = 'hist', color = 'red', bins = 500, alpha = 1)
plt.xlim(0, 3500)

In [ ]:
data.Final_Sales.max()

In [ ]:
data['Final_Sales'].plot(kind = 'hist', color = 'green', bins = 500, alpha =1)
plt.xlim(0, 3500)

In [ ]:
data.RtnMRP.max()

In [ ]:
data['RtnMRP'].plot(kind = 'hist', color = 'blue', bins = 100, alpha = 1)
plt.xlim(0, 800)

In [ ]:
# Pivot the DataFrame Bsaed on SubCat
data_pivoted = data.pivot_table(index = 'SubCat', columns = 'Months', values = 'Quantity')

# Result
data_pivoted.head(10)

# Top Performing drugs

## Based on Quantity

In [ ]:
top_perform_quantity = data.groupby(['DrugName','SubCat','SubCat1'], as_index = False)['Quantity'].sum()
top_perform_quantity.sort_values(by = 'Quantity',ascending= False,inplace = True)
display(top_perform_quantity.head(10))

## Based on Sales

In [ ]:
top_perform_sales = data.groupby(['DrugName','SubCat','SubCat1'], as_index = False)['Final_Sales'].sum()
top_perform_sales.sort_values(by = 'Final_Sales',ascending= False,inplace = True)
display(top_perform_sales.head(10))

## Based On Profit

In [ ]:
data['Profit'] = data['Final_Sales'] - data['Final_Cost']
top_perform_profit = data.groupby(['DrugName','SubCat','SubCat1'], as_index = False)['Profit'].sum()
top_perform_profit.sort_values(by = 'Profit',ascending= False,inplace = True)
display(top_perform_profit.head(10))

In [ ]:
# @title Average Sales by Subcategory

data_pivoted.groupby("SubCat").mean().plot(kind="bar", ylabel="Average Sales Value")

## Data Distribustion

In [ ]:
stats.probplot(data.Quantity, dist = 'norm', plot = pylab)
plt.show()

### Log Transformation

In [ ]:
# Transforming the data to a normal distribution
stats.probplot(np.log(data.Quantity), dist = 'norm', plot = pylab)
plt.show()


## Barplot Quantity of Drug sold by Month


In [ ]:
sns.barplot(data = data, x = 'Week', y = 'Quantity', palette='muted')
plt.title('Quantity of Drugs sold by month')
plt.show()

In **Jan**, **Feb**, **Nov** has the highest Quantity sold. And it is approximatley Same

## Trend in Quantity

In [ ]:
Month = data.groupby('Dateofbill')['Quantity'].sum()
Month

In [ ]:
Month = data.groupby('Months')['Quantity'].sum()
Month = Month.reset_index()

# Create Dictionary to map month names into numeric values
dict_month = {'Jan' : 1,'Feb' : 2,'Mar' : 3,'Apr' : 4,'May' : 5,'Jun' : 6,'Jul' : 7,'Aug' : 8,'Sep' : 9,'Oct' : 10,'Nov' : 11,'Dec' : 12}
Month['MonthNum'] = Month['Months'].map(dict_month)
Month = Month.sort_values('MonthNum')
# Trend Plot
plt.plot(Month['Months'], Month['Quantity'], color = 'blue')
plt.title("Trend in Quantity")
plt.ylabel('Quantity')
plt.xlabel('Month')
plt.show()


In [ ]:
# Assuming 'data' is your DataFrame and 'Dateofbill' is in datetime format
data['Dateofbill'] = pd.to_datetime(data['Dateofbill'])

# Group by week
weekly_data = data.groupby(pd.Grouper(key='Dateofbill', freq='W'))['Quantity'].sum().reset_index()

# Plot the trend
plt.plot(weekly_data['Dateofbill'], weekly_data['Quantity'], color='blue')
plt.title("Trend in Quantity (Weekly)")
plt.ylabel('Quantity')
plt.xlabel('Week')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.tight_layout()  # Adjust the plot to make sure everything fits without overlapping
plt.show()

In [ ]:
# Group by Quantity by Month
grouped = data[['Dateofbill','Quantity']]

df_group = grouped.groupby('Dateofbill').sum()

# Result
df_group.head(10)
df_group.reset_index(inplace = True)
df_group

In [ ]:

# Create new with numerical values of months
df_group['Monthindex'] = df_group['Dateofbill'].map(dict_month)
df_group.sort_values(by = 'Monthindex', inplace = True)
df_group

In [ ]:
df_group.drop('Monthindex', axis = 1, inplace = True)
df_group.reset_index(drop = True, inplace = True)
df_group

### One Hot Encoding

In [ ]:
data1 = pd.get_dummies(df_group.Dateofbill)
data1.columns

In [ ]:
data_clean = pd.concat([df_group, data1], axis = 1)
data_clean

In [ ]:
data_clean["t"] = np.arange(1, 13)
data_clean['t_square'] = data_clean['t'] * data_clean['t']

In [ ]:
data_clean['log_Quantity'] = np.log(data_clean['Quantity'])
data_clean

## Model Building
###Forecasting
#### Data Partition

In [ ]:
Train = data_clean
Test = data_clean

####Linear

In [ ]:
import statsmodels.formula.api as smf

linear = smf.ols('Quantity ~ t', data = Train).fit()

pred_linear = linear.predict(Test['t'])

mape_linear = np.mean(np.abs((Test['Quantity'] - pred_linear)/Test['Quantity'])) * 100

mape_linear


## Exponential

In [ ]:
Exponential = smf.ols('log_Quantity ~ t', data = Train).fit()
pred_exp = Exponential.predict(Test['t'])

mape_Exponential = np.mean(np.abs((Test['Quantity'] - np.exp(pred_exp)) / Test['Quantity'])) * 100

mape_Exponential

### Quadratic

In [ ]:
Quadratic = smf.ols('Quantity ~ t + t_square', data = Train).fit()
pred_Quad = Quadratic.predict(Test[['t', 't_square']])


In [ ]:
mape_Quadratic = np.mean(np.abs((Test['Quantity'] - (pred_Quad))/ Test['Quantity'])) * 100
mape_Quadratic

Additive Seasonality

In [ ]:
add_Seasonality = smf.ols('Quantity ~ Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov+Dec',data = Train).fit()
pred_Seasonality = add_Seasonality.predict(Test[['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']])

In [ ]:
mape_add_Seasonality = np.mean(np.abs((Test['Quantity'] - pred_Seasonality)) / Test['Quantity']) * 100
mape_add_Seasonality

Multiplicative Seasonality

In [ ]:
MulSeasonality = smf.ols('log_Quantity ~ Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov+Dec', data = Train).fit()
pred_MulSeasonality = MulSeasonality.predict(Test)


In [ ]:
mape_Mult_Seasonality = np.mean(np.abs((Test['Quantity'] - np.exp(pred_MulSeasonality))/ Test['Quantity'])) * 100
mape_Mult_Seasonality

Additive Seasonality Quadratic Trend

In [ ]:
Test.columns

In [ ]:
add_SeaQuadratic = smf.ols('Quantity ~ t + t_square+Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov+Dec', data = Train).fit()
pred_add_Sea_Quad = add_SeaQuadratic.predict(Test[[]])